## Introducción al Machine Learning
### Evaluación de modelos de series temporales

En este ejemplo se muestra el efecto de hacer una evaluación train/test o validación cruzada,
haciendo una partición aleatoria o una división temporal de los datos.  En el caso de hacer
una mala división podemos verificar que el modelo sobre-estima el accuracy por estar entrenando
inderectamente por la correlación serial con información que proviene del conjunto de test

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Datos del IBEX35

In [ ]:
import pickle

with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

In [ ]:
tef = stock_data['TEF'].close

In [ ]:
plt.plot(tef)

In [ ]:
from sklearn.preprocessing import scale

Construiremos un modelo para predecir si la rentabilidad a 5 días es positiva.
Utilizaremos las siguientes características
- el mes (estandarizado) 
- dos medias móviles (10 y 50 sesiones) escaladas con la transformación $P/SMA - 1$

In [ ]:
sma10 = tef.rolling(10).mean()
sma50 = tef.rolling(50).mean()
ret5 = tef.pct_change(5)

In [ ]:
p2sma10 = tef/sma10 - 1
p2sma50 = tef/sma50 - 1

In [ ]:
ret5.head(15)

In [ ]:
dataset = pd.DataFrame({
 'month': scale(tef.index.month),   
 'sma10': p2sma10,
 'sma50': p2sma50,
 'target': ret5.shift(-5)    
})

In [ ]:
dataset

Quitamos los NA del principio del final y convertimos la variable objetivo en nominal para simplificar la predicción

In [ ]:
dataset.dropna(inplace=True)
dataset['target'] = (dataset['target'] > 0).astype(str)
dataset

In [ ]:
sns.pairplot(dataset, hue='target')

In [ ]:
features = dataset.drop(columns='target')
target = dataset.target

In [ ]:
target.unique()

In [ ]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, accuracy_score
from sklearn.metrics import confusion_matrix

#### Particion aleatoria usada para datos transversales


In [ ]:
train_x, test_x, train_y, test_y = train_test_split(features,
                                                    target,
                                                    train_size=0.6,
                                                    stratify=target,
                                                    random_state=11
                                                    )

In [ ]:
tree = DecisionTreeClassifier().fit(train_x, train_y)

In [ ]:
pred_y = tree.predict(test_x)

In [ ]:
data_conf_matrix = confusion_matrix(test_y, pred_y, labels=tree.classes_).T
pd.DataFrame(data_conf_matrix, index=tree.classes_, columns=tree.classes_)

In [ ]:
test_y.value_counts()/test_y.value_counts().sum()

In [ ]:
accuracy_score(test_y, pred_y)

Mejoramos algunos puntos a una predicción por defecto, pero este efecto se produce
porque los ejemplos están repartidos aleatoriamente entre train y test.

#### Validación Cruzada con partición lineal.  
En este caso respetamos la relación temporal de los ejemplos
para que no se mezclen en los conjuntos de train/test de forma
consecutiva

In [ ]:
kfold = KFold(n_splits=4, shuffle=False)
acc = []

for train, test in kfold.split(features):
    
    fold_train_x = features.iloc[train]
    fold_train_y = target.iloc[train]
    tree.fit(fold_train_x, fold_train_y)
    
    fold_test_x = features.iloc[test]
    fold_test_y = target.iloc[test]
    fold_pred_y = tree.predict(fold_test_x)
    i_acc = accuracy_score(fold_test_y, fold_pred_y)

    acc.append(i_acc)

El accuracy para cada iteracion

In [ ]:
acc

La estimación del accuracy.  En este mostramos que no es posible encontrar un 
accuracy mejor 

In [ ]:
np.mean(acc)

___

#### Ejercicio propuesto
Probar el efecto de 
- cambiar el parámetro shuffle a True
- Elegir un número distinto de partes (k del CV) 